In [ ]:
from __future__ import division
import pickle
import random
import os
import math
import types
import uuid
import time
from copy import copy
from collections import defaultdict, Counter

import numpy as np

from scipy.stats import binom_test, ttest_1samp

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
import matplotlib as mpl
mpl.rc('savefig', dpi=300)
mpl.rc('text', usetex=True)

In [ ]:
data_dir = os.path.join('data', 'lunarlander-human')

In [ ]:
pilot_ids = ['spike', 'jet', 'faye', 'vicious', 'ed', 'ein', 'julia', 'punch', 'judy', 'lin', 'grencia', 'laughingbull']

In [ ]:
stats_of_pilot = {}

In [ ]:
for pilot_id in pilot_ids:
  with open(os.path.join(data_dir, '%s_pilot_eval.pkl' % pilot_id), 'rb') as f:
    eval_rollouts = list(zip(*(pickle.load(f)[pilot_id])))
  with open(os.path.join(data_dir, '%s_reward_logs.pkl' % pilot_id), 'rb') as f:
    reward_logs = pickle.load(f)
    reward_data = list(reward_logs.values())[0]
  unassisted_succ = [1 if x[1] == 100 else 0 for x in eval_rollouts]
  assisted_succ = [1 if x == 100 else 0 for x in reward_data['outcomes'][0]]
  unassisted_fail = [1 if x[1] == -100 else 0 for x in eval_rollouts]
  assisted_fail = [1 if x == -100 else 0 for x in reward_data['outcomes'][0]]
  stats_of_pilot[pilot_id] = (
    np.mean(unassisted_succ), np.std(unassisted_succ) / np.sqrt(len(unassisted_succ)), len(unassisted_succ),
    np.mean(assisted_succ), np.std(assisted_succ) / np.sqrt(len(assisted_succ)), len(assisted_succ),
    np.mean(unassisted_fail), np.std(unassisted_fail) / np.sqrt(len(unassisted_fail)), len(unassisted_fail),
    np.mean(assisted_fail), np.std(assisted_fail) / np.sqrt(len(assisted_fail)), len(assisted_fail)
  )

In [ ]:
out = []
for pilot_id in pilot_ids:
  s = stats_of_pilot[pilot_id]
  ctrl_succ = s[0]
  ctrl_fail = s[6]
  treat_succ = s[3]
  treat_fail = s[9]
  out.append([pilot_id, 0, ctrl_succ, ctrl_fail])
  out.append([pilot_id, 1, treat_succ, treat_fail])

In [ ]:
# for JMP
with open(os.path.join(data_dir, 'lander_hyp_test.csv'), 'w') as f:
  f.write('userid,assisted,successrate,crashrate\n')
  f.write('\n'.join([','.join([str(z) for z in x]) for x in out]))

In [ ]:
solo_pi_succs = []
solo_pi_crashes = []
comb_succs = []
comb_crashes = []
for pilot_id in pilot_ids:
  s = stats_of_pilot[pilot_id]
  ctrl_succ = s[0]
  ctrl_fail = s[6]
  treat_succ = s[3]
  treat_fail = s[9]
  solo_pi_succs.append(ctrl_succ)
  solo_pi_crashes.append(ctrl_fail)
  comb_succs.append(treat_succ)
  comb_crashes.append(treat_fail)

In [ ]:
mpl.rcParams.update({'font.size': 16})
plt.xlabel('Crash Rate')
plt.ylabel('Success Rate')
plt.title(r'Lunar Lander User Study ($n = %d$)' % len(solo_pi_crashes))
plt.scatter(solo_pi_crashes, solo_pi_succs, label='Solo Human Pilot', color='gray', s=100)
plt.scatter(comb_crashes, comb_succs, label='Human Pilot + Copilot', color='orange', s=100)
plt.scatter([0.156], [0.026], label='Solo Copilot', color='teal', s=100)
plt.legend(loc='upper right')
plt.ylim([-0.05, 1.05])
plt.xlim([-0.05, 1.05])
plt.savefig(os.path.join(data_dir, 'lander-user-study-fig.pdf'), bbox_inches='tight')
plt.show()

In [ ]:
for pilot_id in pilot_ids:
  ctrl_p, ctrl_std, ctrl_n, treat_p, treat_std, treat_n = stats_of_pilot[pilot_id][:6]
  print(
    binom_test(ctrl_p * ctrl_n, ctrl_n, treat_p),
    binom_test(treat_p * treat_n, treat_n, ctrl_p)
  )
  ctrl_p, ctrl_std, ctrl_n, treat_p, treat_std, treat_n = stats_of_pilot[pilot_id][6:]
  print(
    binom_test(ctrl_p * ctrl_n, ctrl_n, treat_p),
    binom_test(treat_p * treat_n, treat_n, ctrl_p)
  )

In [ ]:
unassisted_succ = []
assisted_succ = []
unassisted_fail = []
assisted_fail = []
unassisted_rew = []
assisted_rew = []
for pilot_id in pilot_ids:
  with open(os.path.join(data_dir, '%s_pilot_eval.pkl' % pilot_id), 'rb') as f:
    eval_rollouts = list(zip(*(pickle.load(f)[pilot_id])))
  with open(os.path.join(data_dir, '%s_reward_logs.pkl' % pilot_id), 'rb') as f:
    reward_logs = pickle.load(f)
    reward_data = list(reward_logs.values())[0]
  unassisted_succ.extend([1 if x[1] == 100 else 0 for x in eval_rollouts])
  assisted_succ.extend([1 if x == 100 else 0 for x in reward_data['outcomes'][0]])
  unassisted_fail.extend([1 if x[1] == -100 else 0 for x in eval_rollouts])
  assisted_fail.extend([1 if x == -100 else 0 for x in reward_data['outcomes'][0]])
  unassisted_rew.extend([x[0] for x in eval_rollouts])
  assisted_rew.extend(reward_data['rewards'][0])
stats_of_pilots = (
  np.mean(unassisted_rew), np.std(unassisted_rew) / np.sqrt(len(unassisted_rew)),
  np.mean(assisted_rew), np.std(assisted_rew) / np.sqrt(len(assisted_rew)),
  np.mean(unassisted_succ), np.std(unassisted_succ) / np.sqrt(len(unassisted_succ)),
  np.mean(assisted_succ), np.std(assisted_succ) / np.sqrt(len(assisted_succ)), 
  np.mean(unassisted_fail), np.std(unassisted_fail) / np.sqrt(len(unassisted_fail)), 
  np.mean(assisted_fail), np.std(assisted_fail) / np.sqrt(len(assisted_fail)), 
)

In [ ]:
for i in range(3):
  print('%f \pm %f & %f \pm %f \\\\' % (stats_of_pilots[4*i], stats_of_pilots[4*i+1], stats_of_pilots[4*i+2], stats_of_pilots[4*i+3]))

In [ ]:
binom_test(np.mean(assisted_succ), len(assisted_succ), 0.026)

In [ ]:
binom_test(np.mean(assisted_fail), len(assisted_fail), 0.156)

In [ ]:
print('%f \pm %f & %f & %f & %f \pm %f & %f & %f \\\\' % (
  stats_of_pilots[0], stats_of_pilots[1], stats_of_pilots[4], stats_of_pilots[8],
  stats_of_pilots[2], stats_of_pilots[3], stats_of_pilots[6], stats_of_pilots[10],
))

In [ ]:
solo_outcomes = []
solo_traj = []
assisted_outcomes = []
assisted_traj = []
for pilot_id in pilot_ids:
  with open(os.path.join(data_dir, '%s_pilot_eval.pkl' % pilot_id), 'rb') as f:
    eval_rollouts = pickle.load(f)[pilot_id]
  with open(os.path.join(data_dir, '%s_reward_logs.pkl' % pilot_id), 'rb') as f:
    reward_logs = pickle.load(f)
    reward_data = list(reward_logs.values())[0]
  solo_outcomes.extend(eval_rollouts[1])
  solo_traj.extend(eval_rollouts[2])
  assisted_outcomes.extend(reward_data['outcomes'][0])
  assisted_traj.extend(reward_data['trajectories'][0])

In [ ]:
Counter([t[0][8] for t in solo_traj])

In [ ]:
goals = [round(float(x), 1) for x in np.arange(-0.8, 1, 0.2)]
SUCCESS = 100
CRASH = -100

In [ ]:
def plot_trajectories(outcomes, trajectories, title, file_name=None, G=-0.8, show_goal=True):
  mpl.rcParams.update({'font.size': 16})
  plt.title(title)

  for outcome, trajectory in zip(outcomes, trajectories):
    x, y, vx, vy, a, av, lc, rc, g = list(zip(*trajectory[::5]))[:9]
    if g[0] != G:
      continue
    if outcome == SUCCESS:
      cmap = mpl.cm.YlGn
    elif outcome == CRASH:
      cmap = mpl.cm.YlOrRd
    else:
      cmap = mpl.cm.gray
    plt.scatter(x, y, c=range(len(x)), cmap=cmap, alpha=0.75, linewidth=0)
    if show_goal:
      plt.scatter([g[0]], [0], marker='*', color='yellow', edgecolor='black', linewidth=1, s=300, alpha=0.5)
    
  plt.xlim([-1, 1])
  plt.ylim([-0.1, 1.1])
  plt.xticks([])
  plt.yticks([])
  plt.axis('off')
  if file_name is not None:
    plt.savefig(os.path.join(data_dir, file_name), bbox_inches='tight')
  plt.show()

In [ ]:
plot_trajectories(solo_outcomes, solo_traj, 'Solo Human Pilot', 'human-pilot-solo-traj.pdf')

In [ ]:
plot_trajectories(assisted_outcomes, assisted_traj, 'Human Pilot + Copilot', 'human-pilot-assisted-traj.pdf')

In [ ]:
survey_of_pilot = {
  'vicious': [7, 5, 6, 7, 4, 7, 6, 7, 3, 7, 3, 7, 7]
  'jet': [2, 5, 5, 3, 3, 6, 5, 3, 1, 1, 3, 6, 5]
  'grencia': [6, 6, 6, 5, 5, 4, 7, 6, 5, 5, 5, 6, 6]
  'ed': [1, 7, 6, 4, 6, 6, 5, 4, 2, 3, 4, 7, 7]
  'julia': [2, 7, 5, 4, 3, 6, 6, 4, 5, 6, 3, 6, 6]
  'faye': [5, 6, 5, 5, 4, 5, 5, 6, 6, 6, 7, 7, 5]
  'lin': [3, 6, 7, 4, 3, 4, 7, 5, 4, 7, 6, 7, 7]
  'judy': [5, 2, 7, 5, 6, 7, 7, 1, 6, 7, 3, 7, 7]
  'spike': [4, 5, 7, 7, 5, 3, 7, 7, 5, 4, 5, 7, 7]
  'laughingbull': [5, 2, 4, 2, 3, 3, 6, 4, 2, 4, 6, 7, 3]
  'ein': [5, 7, 6, 5, 5, 4, 6, 3, 2, 2, 5, 7, 7]
  'punch': [7, 7, 7, 6, 3, 6, 7, 4, 6, 6, 6, 7, 7]
}

In [ ]:
survey_questions = [
  'I improved over time',
  'The game was too difficult',
  'The copilot was generally helpful in completing the task',
  'I understood what the copilot was trying to do',
  'I could anticipate the copilot’s behavior',
  'The copilot improved over time',
  'The copilot was helpful in avoiding crashing',
  'The copilot was helpful in avoiding flying out of bounds',
  'The copilot was helpful in landing quickly before time ran out',
  'The copilot was helpful in landing between the flags',
  'The copilot was too aggressive and didn’t give me enough control',
  'I play differently with the copilot than without the copilot',
  'I prefer playing with the copilot'
]

In [ ]:
survey_elts = [[] for _ in range(len(list(survey_of_pilot.values())[0]))]
for k, v in survey_of_pilot.items():
  for j, x in enumerate(v):
    survey_elts[j].append(x)

In [ ]:
survey_ps = [ttest_1samp(x, 4)[1] for x in survey_elts]

In [ ]:
survey_ps

In [ ]:
survey_counts = np.zeros((len(survey_of_pilot), len(list(survey_of_pilot.values())[0]), 7))
for i, (k, v) in enumerate(survey_of_pilot.items()):
  for j, x in enumerate(v):
    survey_counts[i, j, (8-x)-1] += 1

In [ ]:
survey_counts = survey_counts.sum(axis=0)

In [ ]:
survey_counts

In [ ]:
question_avgs = (survey_counts * np.tile(np.arange(1, 8, 1), survey_counts.shape[0]).reshape(survey_counts.shape))
question_avgs = question_avgs.sum(axis=1) / len(pilot_ids)

In [ ]:
question_avgs

In [ ]:
def fmt_pval(p):
  if p < 0.001:
    return '<0.001'
  elif p < 0.01:
    return '<0.01'
  elif p < 0.05:
    return '<0.05'
  else:
    return '>0.05'

In [ ]:
for i, x in enumerate(survey_questions):
  if survey_ps[i] < 0.05 and question_avgs[i] > 4:
    print('& \\textbf{%s} & $\\mathbf{%s}$ & $\\mathbf{%0.2f}$ & %d & %d & %d & %d & %d & %d & %d \\\\' % (x, fmt_pval(survey_ps[i]), round(question_avgs[i], 2), *list(survey_counts[i])))
  else:
    print('& %s & $%s$ & %0.2f & %d & %d & %d & %d & %d & %d & %d \\\\' % (x, fmt_pval(survey_ps[i]), round(question_avgs[i], 2), *list(survey_counts[i])))